In [1]:
!pip install faiss-gpu langchain transformers
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is

In [2]:
import os

def split_file(input_file, output_dir, num_chunks):
    os.makedirs(output_dir, exist_ok=True)
    file_size = os.path.getsize(input_file)
    chunk_size = file_size // num_chunks
    current_chunk = 1
    current_chunk_size = 0

    with open(input_file, 'r', encoding='utf-8') as file:
        chunk_file = open(os.path.join(output_dir, f'chunk_{current_chunk}.txt'), 'w', encoding='utf-8')
        for line in file:
            if current_chunk_size + len(line) > chunk_size and current_chunk < num_chunks:
                chunk_file.close()
                current_chunk += 1
                current_chunk_size = 0
                chunk_file = open(os.path.join(output_dir, f'chunk_{current_chunk}.txt'), 'w', encoding='utf-8')
            chunk_file.write(line)
            current_chunk_size += len(line)
        chunk_file.close()


input_file = '/kaggle/input/sogang-nlp-rag/processed_wikipedia.txt/processed_wikipedia.txt'
output_dir = '/kaggle/working/chunks'
num_chunks = 100
split_file(input_file, output_dir, num_chunks)


In [3]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
import os

def create_faiss_index_from_file(file_path, model, index_path, nlist=100, m=8):
    # Load the text data
    with open(file_path, 'r', encoding='utf-8') as file:
        texts = file.readlines()
    
    # Generate embeddings using GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    embeddings = model.encode(texts, convert_to_tensor=True, device=device)
    embeddings = embeddings.cpu().numpy()  # Move to CPU before passing to FAISS
    
    # Create quantized FAISS index with GPU support
    dimension = embeddings.shape[1]
    res = faiss.StandardGpuResources()
    
    # Configure GpuIndexIVFPQ
    config = faiss.GpuIndexIVFPQConfig()
    config.device = 0  # Assumes using the first GPU
    
    # Constructing the GpuIndexIVFPQ
    gpu_index = faiss.GpuIndexIVFPQ(res, dimension, nlist, m, 8, faiss.METRIC_L2, config)
    
    # Training the index
    gpu_index.train(embeddings)
    gpu_index.add(embeddings)
    
    # Convert GPU index to CPU index before saving
    cpu_index = faiss.index_gpu_to_cpu(gpu_index)
    
    # Save the index
    faiss.write_index(cpu_index, index_path)
    

# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create indexes for each chunk
chunk_dir = '/kaggle/working/chunks'
index_dir = '/kaggle/working/faiss_indexes'
os.makedirs(index_dir, exist_ok=True)

for chunk_file in os.listdir(chunk_dir):
    chunk_path = os.path.join(chunk_dir, chunk_file)
    # Remove the .txt extension for the index file name
    base_name = os.path.splitext(chunk_file)[0]
    index_path = os.path.join(index_dir, f'{base_name}.index')
    create_faiss_index_from_file(chunk_path, model, index_path)
    print(f'Index for chunk {chunk_file} created.')

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-13 09:30:01.245819: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 09:30:01.245941: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 09:30:01.382244: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Index for chunk chunk_78.txt created.


Batches:   0%|          | 0/32134 [00:00<?, ?it/s]

Index for chunk chunk_85.txt created.


Batches:   0%|          | 0/38692 [00:00<?, ?it/s]

Index for chunk chunk_8.txt created.
